### Day 5

We have a series of coordinates that define lines on a grid. We're asked to find the number of intersection points between all the vertical and horizontal lines

In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
with open('d5.txt') as file:
    puzzle_input = file.readlines()

puzzle_input[:5]

['527,299 -> 430,299\n',
 '828,228 -> 81,228\n',
 '40,126 -> 842,928\n',
 '30,390 -> 30,741\n',
 '196,853 -> 196,599\n']

Let's see if we can do that fancy regex unpacking thing

In [2]:
pattern = re.compile('([0-9]+),([0-9]+) -> ([0-9]+),([0-9]+)')
def extract_coordinates(string):
    
    matches = re.match(pattern, string)    
   
    return pd.DataFrame({
        'x1' : int(matches.group(1)),
        'y1' : int(matches.group(2)),
        'x2' : int(matches.group(3)),
        'y2' : int(matches.group(4))
    }, index = [0])

coordinates = reduce(
    lambda x, y : pd.concat([x, y]),
    map(extract_coordinates, puzzle_input)
)

coordinates = (
    coordinates
    .reset_index(drop = True)
    .reset_index(drop = False)
    .rename(columns = {'index' : 'id'})
)
coordinates

,id,x1,y1,x2,y2
0,0,527,299,430,299
1,1,828,228,81,228
2,2,40,126,842,928
3,3,30,390,30,741
4,4,196,853,196,599
...,...,...,...,...,...
495,495,825,689,873,641
496,496,656,191,134,191
497,497,862,818,78,34
498,498,22,952,961,13


Now we're asked to only look at "straight" lines so we can shorten this list to ones where either x1 == x2 or y1 == y2

In [3]:
coordinates_strt = coordinates.query('(x1 == x2) | (y1 == y2)')
coordinates_strt

,id,x1,y1,x2,y2
0,0,527,299,430,299
1,1,828,228,81,228
3,3,30,390,30,741
4,4,196,853,196,599
5,5,388,795,388,300
...,...,...,...,...,...
491,491,140,920,574,920
492,492,639,873,585,873
494,494,117,691,117,158
496,496,656,191,134,191


I initially had the idea to do this just by calculations of comparing points and not actually make a representation of the grid but it got real complicated, real fast. We'll do the numpy array method since that is something I wanted to practice anyways

In [4]:
# Start by makin a grid
xlen = max([coordinates_strt.x1.max(), coordinates_strt.x2.max()]) + 1
ylen = max([coordinates_strt.y1.max(), coordinates_strt.y2.max()]) + 1

grid = np.zeros(
    shape = [ylen, xlen],
    dtype = 'int'
)

# now loop through and mark:
for i in range(len(coordinates_strt)):
    row = coordinates_strt.iloc[i,]
    x1 = row[1]
    y1 = row[2]
    x2 = row[3]
    y2 = row[4]
    
    # Vertical line:
    if x1 == x2:
        for i in range(min(y1, y2), max(y1, y2) + 1):
            grid[i][x1] += 1
    # Horizontal line:
    else:
        for i in range(min(x1, x2), max(x1, x2) + 1):
            grid[y1][i] += 1

np.where(
    grid > 1,
    1,
    0
).sum()

8111

### Part 2:

Now we have to do the same thing but with the diagonals which we're told only occur at 45 degree angles which is not too bad. Basically just have to incorporate that into our marking:

In [5]:
# Start by makin a grid
xlen = max([coordinates.x1.max(), coordinates.x2.max()]) + 1
ylen = max([coordinates.y1.max(), coordinates.y2.max()]) + 1

grid = np.zeros(
    shape = [ylen, xlen],
    dtype = 'int'
)

# now loop through and mark:
for i in range(len(coordinates)):
    row = coordinates.iloc[i,]
    x1 = row[1]
    y1 = row[2]
    x2 = row[3]
    y2 = row[4]
    
    # Vertical line:
    if x1 == x2:
        for i in range(min(y1, y2), max(y1, y2) + 1):
            grid[i][x1] += 1
    # Horizontal line:
    elif y1 == y2:
        for i in range(min(x1, x2), max(x1, x2) + 1):
            grid[y1][i] += 1
    # Diagonal line
    else:
        # Let's move from x1, y1 to x2, y2
        # Figure out if we're going up or down, left to right:
        if x1 > x2:
            x_dir = -1
        else:
            x_dir = 1
        if y1 > y2:
            y_dir = -1
        else:
            y_dir = 1
        
        distance = abs(x1 - x2) + 1
        for i in range(distance):
            grid[y1 + i*y_dir][x1 + i*x_dir] += 1
        

np.where(
    grid > 1,
    1,
    0
).sum()

22088